# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

Задача - Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества F1 не меньше 0.75. 

Алгоритм решения выглядит так:

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Выводы.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.pipeline import Pipeline
import spacy
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [3]:
toxic_comments.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
#Посмотрим, сколько у нас токсичных/нетоксичных текстов
toxic_count = toxic_comments['toxic'].value_counts()
display(toxic_count)

#Выведем соотношение
class_ratio = toxic_count[0] / toxic_count[1]
class_ratio

0    143106
1     16186
Name: toxic, dtype: int64

8.841344371679229

**V2** 

In [5]:
tqdm.pandas()

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def lemmatize_text(text):
    doc = nlp(text)
    lemm_text = " ".join([token.lemma_ for token in doc])
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text)
    return cleared_text.lower()

toxic_comments['lemm_text'] = toxic_comments['text'].progress_apply(lemmatize_text)
toxic_comments.drop(['text'], axis=1, inplace=True)


100%|██████████| 159292/159292 [20:15<00:00, 131.03it/s]


In [6]:
toxic_comments.head()

,Unnamed: 0,toxic,lemm_text
0,0,0,explanation why the edit make under my usern...
1,1,0,d aww he match this background colour i be s...
2,2,0,hey man i be really not try to edit war it...
3,3,0,more i can not make any real suggestion ...
4,4,0,you sir be my hero any chance you rememb...


In [7]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   toxic       159292 non-null  int64 
 2   lemm_text   159292 non-null  object
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [8]:
target = toxic_comments['toxic']
features = toxic_comments['lemm_text']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)


nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

print(features_train.shape)
print(features_test.shape)

cv_counts = 3


(127433,)
(31859,)


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Обучение

**Классификатор LogisticRegression**

In [9]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('classifier', LogisticRegression(class_weight='balanced'))
])

# Определение гиперпараметров для поиска по сетке
hyperparams = {
    'tfidf__stop_words': [stopwords],
    'classifier__C': [5, 10, 15]
}

# Поиск по сетке с использованием кросс-валидации
grid_search = GridSearchCV(pipeline, hyperparams, scoring='f1', cv=cv_counts)
grid_search.fit(features_train, target_train)

# Вычисление F1-меры через grid_search.best_score_
cv_f1_LR = grid_search.best_score_
print("Лучшая F1-мера на обучающей выборке:", cv_f1_LR)

Лучшая F1-мера на обучающей выборке: 0.759828504660053


**Классификатор DecisionTreeClassifier**

In [10]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('classifier', DecisionTreeClassifier(class_weight='balanced'))
])

# Определение гиперпараметров для поиска по сетке
hyperparams = {
    'tfidf__stop_words': [stopwords],
    'classifier__max_depth': [x for x in range(1, 11, 2)],
    'classifier__random_state': [12345]
}

# Поиск по сетке с использованием кросс-валидации
grid_search = GridSearchCV(pipeline, hyperparams, scoring='f1', cv=cv_counts)
grid_search.fit(features_train, target_train)

# Вывод лучших параметров
print("Лучшие параметры, найденные на обучающей выборке:")
print(grid_search.best_params_)
print()

# Вычисление F1-меры для лучших параметров
cv_f1_DTC = grid_search.best_score_
print("Лучшая F1-мера на обучающей выборке:", cv_f1_DTC)

Лучшие параметры, найденные на обучающей выборке:
{'classifier__max_depth': 9, 'classifier__random_state': 12345, 'tfidf__stop_words': {'have', 'into', 'do', 'few', 'over', 'some', 'the', 'weren', 'yourselves', 'himself', 'with', 'own', "you're", 'should', 'has', "you've", 'our', "she's", 'up', 'is', 'doesn', 'off', 'because', 'themselves', 'again', 'an', 'above', 'under', 'through', 'are', "doesn't", "hasn't", 'll', 'their', 'while', 'why', 'his', "don't", 'mightn', 'doing', 'during', 'had', 'whom', 'am', 'below', 'further', 'no', "didn't", 'down', 'shan', 'this', 'of', 'now', 'ours', 'd', "couldn't", 't', 'me', 'couldn', 'ourselves', 'your', 'a', 'very', 'same', "shan't", 'needn', 'to', 'shouldn', "isn't", 'were', 're', "it's", 'who', 'where', 'y', 'nor', 'which', 'itself', "should've", 'in', 'or', 'wasn', 'was', "mightn't", "wouldn't", 'at', 'once', 'those', 'm', 'both', 'more', 'be', 'her', 'hadn', 'but', 'other', "you'd", "hadn't", 'too', 'as', 'all', 's', 'theirs', 'between', 'wi

**Классификатор RandomForestClassifier**

In [11]:

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('classifier', RandomForestClassifier(random_state=12345, class_weight='balanced'))
])


hyperparams = {
    'tfidf__stop_words': [stopwords],
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [2, 5, 10]
}


grid_search = GridSearchCV(pipeline, hyperparams, scoring='f1', cv=cv_counts)
grid_search.fit(features_train, target_train)


cv_f1 = grid_search.best_score_

print("F1-мера на кросс-валидации:", cv_f1)

F1-мера на кросс-валидации: 0.3732003199151462


In [12]:
index = ['LogisticRegression',
         'DecisionTreeClassifier',
         'RandomForestClassifier']
data = {'F1 на CV':[cv_f1_LR,
                    cv_f1_DTC,
                    cv_f1]
        }

scores_data = pd.DataFrame(data=data, index=index)
scores_data['Выполнение задачи'] = scores_data['F1 на CV'] > 0.75
scores_data

,F1 на CV,Выполнение задачи
LogisticRegression,0.759829,True
DecisionTreeClassifier,0.565652,False
RandomForestClassifier,0.373200,False


**V3** <div class="alert" style="background-color:#ead7f7;color:#8737bf">
    <font size="3">Из всех нам подходит только один - LogisticRegression </font>
   
</div>

In [13]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('classifier', LogisticRegression(class_weight='balanced'))
])

# Определение гиперпараметров для поиска по сетке
hyperparams = {
    'tfidf__stop_words': [stopwords],
    'classifier__C': [5, 10, 15]
}

# Поиск по сетке с использованием кросс-валидации
grid_search = GridSearchCV(pipeline, hyperparams, scoring='f1', cv=cv_counts)
grid_search.fit(features_train, target_train)
best_model = grid_search.best_estimator_

predictions = best_model.predict(features_test)
test_f1 = f1_score(target_test, predictions)
print("F1-мера на тестовой выборке:", test_f1)

F1-мера на тестовой выборке: 0.7724731182795699


В данной таблице представлены значения метрики F1 на кросс-валидации (CV) и на валидационной выборке для каждой модели. Кроме того, добавлен столбец "Выполнение задачи", который указывает, выполнена ли задача (значение F1 на валидации > 0.75). В данном случае, только модель LogisticRegression выполняет поставленную задачу, так как ее F1 на валидации превышает пороговое значение 0.75.

## Выводы

 **V3** <div class="alert" style="background-color:#ead7f7;color:#8737bf">
    <font size="3">
В ходе работы были выполнены следующие шаги:

1. Подготовка данных: были подготовлены данные для обучения моделей.

2. Балансировка классов: выбран способ балансировки классов в моделях 'balanced', и данные были разделены на обучающую и тестовую выборки.

3. Обучение моделей: были обучены различные модели, включая LogisticRegression, DecisionTreeClassifier и RandomForest.

Оценка качества моделей: на основе валидационной выборки была выбрана лучшая модель по f1 CV. И протестирована на тестовой выборке.

В результате анализа, лучшей моделью оказалась LogisticRegression. CatBoostClassifier может показать высокое качество при длительном обучении на большом объеме данных.